In [430]:
import pandas as pd

In [431]:
ratings = pd.read_csv('ml-latest-small\\ratings.csv')
movies = pd.read_csv('ml-latest-small\\movies.csv')
queries = pd.read_csv('keywords.csv')

Задание 1

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [432]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

for region in geo_data:
    geo_data[region] = set(geo_data[region])

queries.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [433]:
def region_def(row, dct):
    '''Возвращает название региона, если поисковый запрос содержит название города региона.
    Если поисковый запрос не содержит названия города, то возвращается 'undefined'.'''
    return next(region if any(word in cities for word in row.lower().split())
                else 'undefined' for region, cities in dct.items())
    
queries['region'] = queries['keyword'].apply(region_def, dct=geo_data)

queries[~(queries['region']=='undefined')].head()

,keyword,shows,region
127,авито москва,979292,Центр
849,авито ярославль,209581,Центр
1259,москва,153050,Центр
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр


In [434]:
queries.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


Задание 2

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [435]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [436]:
avg_rating = ratings[['movieId', 'rating']].groupby('movieId').mean().reset_index()
avg_rating = avg_rating.rename(columns = {'rating': 'avg_rating'})

def rating_class(row):
    if row <= 2.0:
        return 'низкий рейтинг'
    elif row <= 4.0:
        return 'средний рейтинг'
    elif row <= 5.0:
        return 'высокий рейтинг'
    
avg_rating['class'] = avg_rating['avg_rating'].apply(rating_class)

In [437]:
avg_rating.head()

,movieId,avg_rating,class
0,1,3.872470,средний рейтинг
1,2,3.401869,средний рейтинг
2,3,3.161017,средний рейтинг
3,4,2.384615,средний рейтинг
4,5,3.267857,средний рейтинг


In [438]:
ratings = pd.merge(ratings, avg_rating, how='left', on='movieId').sort_values(['userId', 'movieId'])
ratings.head()

,userId,movieId,rating,timestamp,avg_rating,class
0,1,31,2.5,1260759144,3.178571,средний рейтинг
1,1,1029,3.0,1260759179,3.702381,средний рейтинг
2,1,1061,3.0,1260759182,3.545455,средний рейтинг
3,1,1129,2.0,1260759185,3.312500,средний рейтинг
4,1,1172,4.0,1260759205,4.260870,высокий рейтинг


In [439]:
movies_ratings = pd.merge(movies, avg_rating, how='left', on='movieId').sort_values(['movieId'])
movies_ratings.head()

,movieId,title,genres,avg_rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,средний рейтинг


Задание 3

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [440]:
ratings.head()

,userId,movieId,rating,timestamp,avg_rating,class
0,1,31,2.5,1260759144,3.178571,средний рейтинг
1,1,1029,3.0,1260759179,3.702381,средний рейтинг
2,1,1061,3.0,1260759182,3.545455,средний рейтинг
3,1,1129,2.0,1260759185,3.312500,средний рейтинг
4,1,1172,4.0,1260759205,4.260870,высокий рейтинг


In [441]:
top_user = ratings[['userId', 'rating']].groupby('userId').count().reset_index()
top_user_lst = top_user[top_user['rating']>=100]['userId'].tolist()

ratings_top_user = ratings[ratings['userId'].isin(top_user_lst)].groupby('userId').agg({'timestamp': [min, max]})

ratings_top_user['Lifetime, days'] = ratings_top_user.apply(
    lambda row: (row['timestamp']['max']-row['timestamp']['min'])/(3600*24), axis=1)

In [442]:
ratings_top_user[['Lifetime, days']].sort_values('Lifetime, days', ascending=False).head(10)

,"Lifetime, days"
,
userId,
380,5969.787743
547,5807.992303
427,5515.847870
15,5455.943241
624,5295.052164
529,5189.035347
648,3730.094144
388,3269.892303


Задание 4

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [443]:
years = set(map(str, range(1950, 2011)))

In [444]:
import re
def extract_year(row):
    match = re.search('\(\D*\d{4}\D*\)', row)
    if match:
        year = re.search('\d{4}', match[0])[0]
        if year in years:
            return year
    return 1900

In [445]:
movies['year'] = movies['title'].apply(extract_year)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [446]:
#avg_rating - из задания №2
movies = pd.merge(movies, avg_rating[['movieId', 'avg_rating']], how='left', on='movieId')
movies.groupby('year').mean().sort_values('avg_rating', ascending=False)[['avg_rating']].head(10)

,avg_rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295


In [287]:
#P.S.: раньше было лучше, но это не точно